In [ ]:
import json
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [ ]:
db = load_db()
cursor = db.cursor()

In [ ]:
include_place = [22,179,10,59,200]
total_path = [22,179,10,1,2,3,4,5,6,7,8,9]
id = 2200
sql1 = f"SELECT places FROM course_ WHERE {id} = ANY(places)"
sql2 = f"select places from course_ where array{include_place}::smallint[] <@ places"
cursor.execute(sql2)
result = cursor.fetchall()
if len(result) == 0:
  print("empty")
else:
  print(result)


[([142, 63, 59, 98, 8, 15, 17, 22, 174, 179, 10, 200, 21, 4, 13, 46],), ([1, 21, 42, 8, 52, 3, 29, 9, 214, 17, 185, 387, 10, 240, 66, 60, 22, 179, 903, 78, 59, 31, 200, 8, 80],)]


In [ ]:
for i in result:
  print(i)

([142, 63, 59, 98, 8, 15, 17, 22, 174, 179, 10, 200, 21, 4, 13, 46],)
([1, 21, 42, 8, 52, 3, 29, 9, 214, 17, 185, 387, 10, 240, 66, 60, 22, 179, 903, 78, 59, 31, 200, 8, 80],)


In [ ]:
my_list = [80,27,10]
sql2 = f"select places from course_ where array{my_list}::smallint[] <@ places"
cursor.execute(sql2)
result = cursor.fetchall()
for i in result:
  print(result)

[([80, 105, 65, 38, 27, 10, 20, 481, 16, 49, 5, 214, 15, 9, 3, 52, 13, 25],)]


In [ ]:
#중첩 쿼리? 
a1 = [1,2,3,4,5]
a2 = [1,2,3,4,5,6]


TypeError: ignored

##combination checking

In [ ]:
from itertools import combinations
min_length = 3
max_length = 5
length = len(my_list)
load_course_sql= (f'SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,\
                    places FROM course_ WHERE length>={min_length} and length<={max_length}')
while length > i:
  possible_cases = list(combinations[my_list],length-i)
  ##sql query 날리고 결과 받아서
  for now in possible_cases:
    query_result = "sample query"
    ##cursor.execute~~~~ result~~~
    result = "33"
    if len(result) == 0:
      continue
    else:
      print("이걸로 쿼리해서 없는것 찾아낸 다음에 break 해야겠지??")
      break
  i-=1




[]

In [ ]:
min_length = 3
max_length = 5
length = len(my_list)
load_course_sql= (f"SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places\
 FROM course_ WHERE length>={min_length} and length<={max_length} and array{my_list}::smallint[] <@ places")
#위 query로 받은 places와 my_list 비교 후 없는 거는 집어넣어야함. 이때 length가 수정 될 것.


print(load_course_sql)

SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places FROM course_ WHERE length>=3 and length<=5


In [ ]:
include = [1,2,3,4,5]
comb_result = list(combinations(include,4))
type(comb_result[0])

tuple

In [ ]:
count = 0
length = len(include)
while length > count:
  possible_cases = list(combinations(include,length-count))
  for now in possible_cases:
    print(list(now))
    
  
  print("*****"*10)
  count+=1

[1, 2, 3, 4, 5]
**************************************************
[1, 2, 3, 4]
[1, 2, 3, 5]
[1, 2, 4, 5]
[1, 3, 4, 5]
[2, 3, 4, 5]
**************************************************
[1, 2, 3]
[1, 2, 4]
[1, 2, 5]
[1, 3, 4]
[1, 3, 5]
[1, 4, 5]
[2, 3, 4]
[2, 3, 5]
[2, 4, 5]
[3, 4, 5]
**************************************************
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[2, 3]
[2, 4]
[2, 5]
[3, 4]
[3, 5]
[4, 5]
**************************************************
[1]
[2]
[3]
[4]
[5]
**************************************************


##course loading 함수

In [ ]:
def load_course(day,db,include):
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  length = len(include)
  count = 0
  
  df = pd.DataFrame()
  while length > count:
    possible_cases = list(combinations(include,length-count))
    ##sql query 날리고 결과 받아서
    for now in possible_cases:
      print("now : ",list(now))
      now_include = list(now)
      load_course_sql= (f"SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places\
 FROM course_ WHERE length>={min_length} and length<={max_length} and array{now_include}::smallint[] <@ places")
      #해당 query문으로 query 날려서 결과가 있는지 확인해야함.
      
      cursor.execute(load_course_sql)
      result = cursor.fetchall()

      #현재 포함시킬 place를 모두 가지는 코스가 없는경우 
      if len(result) == 0:
        continue
      #현재 Place를 모두 포함하는 코스가 있는경우
      else:
        print("현재 now를 포함하는 df : \n",pd.DataFrame(result),"\n****************************\n")
        df = pd.concat([df, pd.DataFrame(result)])
    
    #dataframe에 값이 존재하면 (최적의 코스를 찾았다면)
    if not df.empty:
      break    
    count += 1

  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places']

  return df
  #intersect 밖의 place를 모든 df의 Places list에 append 해줘야함.

##df concat checking

In [ ]:
tmp_df = pd.DataFrame()
tmp_df.empty

True

In [ ]:
course_data = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/07.31_course_db.csv")
sample_course = course_data.sample()

In [ ]:
concat = pd.concat([tmp_df,sample_course])
concat

,Unnamed: 0,id,Package Title,list_path,path_only_id,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,len
353,353,354,뚜벅이로 제주 보름살기,"['월정리해변', '세화항', '해녀박물관', '월정리해변', '제주올레20코스',...","[13, 915, 111, 13, 395, 3, 52, 346, 6, 42, 13,...",2.210526,2.526316,1.684211,1.684211,2.210526,0.736842,2.947368,0.578947,8,19


In [ ]:
# wonseok = pd.DataFrame((['wonseok','male','25']))
# wonseok
wonseok = tuple([['wonseok','male','25']])
wonsoek = pd.DataFrame(wonseok)
wonsoek

,0,1,2
0,wonseok,male,25


In [ ]:
empty_list = []
check_empty = pd.DataFrame(empty_list)
check_empty.empty

True

In [ ]:
include_list = [110,10,16,17,20]
day = 4
db = load_db()
my_course = load_course(day,db,include_list)

now :  [110, 10, 16, 17, 20]
now :  [110, 10, 16, 17]
sql :  SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places FROM course_ WHERE length>=8 and length<=16 and array[110, 10, 16, 17]::smallint[] <@ places
[]
now :  [110, 10, 16, 20]
now :  [110, 10, 17, 20]
now :  [110, 16, 17, 20]
now :  [10, 16, 17, 20]
now :  [110, 10, 16]
result :  [(1251, '제주에 푹 담궈진 2박3일', 2.083333333333333, 2.4166666666666665, 1.5833333333333333, 1.4166666666666667, 2.4166666666666665, 0.75, 2.75, 0.5833333333333334, 0, [110, 10, 153, 16, 92, 270, 9, 9, 3, 13, 25, 18]), (2814, '제주도 겨울여행', 1.9375, 2.4375, 1.625, 1.0625, 2.125, 0.875, 2.625, 0.3125, 3, [202, 22, 93, 16, 10, 58, 18, 32, 110, 4, 132, 112, 21, 2, 29, 9]), (2850, '제주도 3박 4일', 2.1, 2.4, 1.2, 1.5, 2.0, 0.8, 2.3, 0.4, 1, [18, 13, 46, 2, 5, 16, 110, 22, 10, 8])]
현재 now를 포함하는 df : 
      0               1         2         3         4         5         6   \
0  1251  제주에 푹 담궈진 2박3일  2.083333  2.416667  1.583333  1.416667 

In [ ]:
my_course

,id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places
0,1251,제주에 푹 담궈진 2박3일,2.083333,2.416667,1.583333,1.416667,2.416667,0.750000,2.750000,0.583333,0,"[110, 10, 153, 16, 92, 270, 9, 9, 3, 13, 25, 18]"
1,2814,제주도 겨울여행,1.937500,2.437500,1.625000,1.062500,2.125000,0.875000,2.625000,0.312500,3,"[202, 22, 93, 16, 10, 58, 18, 32, 110, 4, 132,..."
2,2850,제주도 3박 4일,2.100000,2.400000,1.200000,1.500000,2.000000,0.800000,2.300000,0.400000,1,"[18, 13, 46, 2, 5, 16, 110, 22, 10, 8]"
0,1635,쌍둥이와 2박3일,1.500000,2.000000,1.400000,1.200000,1.600000,1.100000,2.200000,1.100000,10,"[217, 59, 8, 110, 10, 20, 15, 18, 110, 110]"
0,359,뚜벅이 커플의 3박4일 제주도일주,2.307692,2.538462,1.538462,1.615385,2.000000,0.461538,2.615385,0.538462,4,"[3, 9, 17, 5, 16, 10, 197, 38, 60, 24, 8, 31, 37]"
1,494,놀멍쉬멍 봄 제주??,2.250000,2.500000,1.083333,1.500000,1.750000,0.833333,2.416667,0.166667,1,"[59, 341, 10, 5, 573, 22, 903, 16, 112, 17, 37..."
2,587,2015 JEJU TRAVEL : D,2.428571,2.500000,1.857143,1.142857,2.357143,0.500000,2.428571,0.357143,4,"[8, 179, 10, 24, 5, 65, 16, 17, 9, 3, 219, 279..."
3,938,3박4일 제주도 헠,2.454545,2.727273,1.545455,1.636364,2.181818,0.454545,2.545455,0.090909,4,"[13, 9, 2, 17, 5, 16, 768, 24, 10, 174, 8]"
4,962,제주 with 성찬,2.444444,2.777778,1.777778,1.666667,2.444444,0.666667,2.444444,0.222222,8,"[9, 3, 13, 1, 17, 16, 10, 8, 59]"
5,1045,[성민서현] 제주도 16.11.06,2.500000,2.416667,1.416667,1.250000,1.750000,0.583333,2.416667,0.500000,4,"[37, 6, 3, 9, 24, 22, 10, 16, 5, 17, 15, 83]"


In [ ]:
pd.np.empty((0, 4))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.


array([], shape=(0, 4), dtype=float64)

In [ ]:
load_course_sql= (f"SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places\
 FROM course_ WHERE length>={5} and length<={24} and array{[110,10,16,17]}::smallint[] <@ places")
      #해당 query문으로 query 날려서 결과가 있는지 확인해야함.
print(load_course_sql)

SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places FROM course_ WHERE length>=5 and length<=24 and array[110, 10, 16, 17]::smallint[] <@ places


In [ ]:
cursor.execute(load_course_sql)
result = cursor.fetchall()
print(result)

[(996, '부모님과 제주투어', 1.8235294117647056, 2.0588235294117645, 1.3529411764705883, 1.3529411764705883, 1.8823529411764703, 0.8823529411764706, 2.647058823529412, 0.4117647058823529, 3, [155, 22, 24, 233, 10, 16, 17, 41, 18, 83, 29, 9, 4, 2, 6, 13, 110]), (3096, '제주도', 1.7142857142857142, 2.142857142857143, 1.4285714285714286, 1.4285714285714286, 1.761904761904762, 1.2857142857142858, 2.571428571428572, 0.5238095238095238, 11, [355, 10, 110, 13, 61, 1, 15, 62, 31, 233, 319, 85, 22, 2, 63, 165, 223, 8, 16, 3, 17])]


In [ ]:
[110,10,16,17] ==[110,10,16,17]

True

#버그 fix 및 Lambda 업로드용으로 수정

###lambda 업로드함수

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

def calculate_taste(user_id,db):
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM tourist_attraction WHERE id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    print('place : ',place,"\tnow : ",place_theme)

  result = theme_score / len(survey_result)
  return result


def path_divider(day,path):
  path_per_day = []
  checker = day

  length = len(path)
  print('length :',length )

  remainder = length%day
  quotient = length//day

  start = 0
  check=0

  while start<length:
    print(path[start:start + quotient + check])
    path_per_day.append(path[start:start + quotient + check])
    start += quotient+check
    checker-=1
    if checker<=remainder:
      check = 1

  return path_per_day


def load_course(day,db,include):
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  length = len(include)
  count = 0
  
  df = pd.DataFrame()
  while length >= count:
    possible_cases = list(combinations(include,length-count))
    ##sql query 날리고 결과 받아서
    for now in possible_cases:
      print("now : ",list(now))
      now_include = list(now)
      load_course_sql= (f"SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places\
 FROM course_ WHERE length>={min_length} and length<={max_length} and array{now_include}::smallint[] <@ places")
      #해당 query문으로 query 날려서 결과가 있는지 확인해야함.
      
      cursor.execute(load_course_sql)
      result = cursor.fetchall()

      #현재 포함시킬 place를 모두 가지는 코스가 없는경우 
      if len(result) == 0:
        continue
      #현재 Place를 모두 포함하는 코스가 있는경우
      else:
        print("현재 now를 포함하는 df : \n",pd.DataFrame(result),"\n****************************\n")
        df = pd.concat([df, pd.DataFrame(result)])
    
    #dataframe에 값이 존재하면 (최적의 코스를 찾았다면)
    if not df.empty:
      break    
    count += 1
  print('df : ',df)
  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places']

  return df
  #intersect 밖의 place를 모든 df의 Places list에 append 해줘야함.

  
  
  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places']

  return df
  #intersect 밖의 place를 모든 df의 Places list에 append 해줘야함.


def recommend_by_theme(user_id, day, include_list):
  top = 5
  db = load_db()
  course = load_course(day,db,include_list)
  user_taste = calculate_taste(user_id,db)
  user_taste = user_taste.reshape(1,-1)

  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

  print(user_taste)
  
  similarity = cosine_similarity(user_taste, course[tag])
  similarity = similarity[0]
  print(similarity.shape)
      
  print('similarity : ',similarity)
  
  result_index = similarity.argsort()
  result_index = result_index
  
  print("result_index : ",result_index)
#line61~64 확인용
  for i in range(-1,-(len(result_index)+1),-1):
    now_idx = result_index[i]
    sim = similarity[now_idx]
    print("similarity : ",sim,"df index : ",now_idx)

  top_N_result = result_index[-1:-(top+1):-1]
  print("top_N_result",top_N_result)
  
  #결과는 가장 일치율이 높은거 1개의 index만 던져주자
  result = top_N_result[0]
  result_df = course.iloc[result]
  print(result_df)
  not_included_places = list(set(include_list) - set(result_df['places']))

  result_df['places'] += not_included_places
  path = path_divider(day,result_df['places'])
  print('path : ',path)
  return path

In [ ]:

user_id = 187
day = 5
include_list = [0]
result = recommend_by_theme(user_id,day,include_list)

now :  [0]
now :  []
현재 now를 포함하는 df : 
         0                                     1         2         3   \
0       41                         싱싱싱의 3박 4일 제주  1.900000  2.100000   
1       44                           뚝의 제주 2박 3일  2.500000  2.700000   
2       46                      제주도 3박 4일 뚜벅이 여행  2.363636  2.636364   
3       47                    나홀로 제주도 버스여행 3박 4일  2.333333  2.400000   
4       49  [한국/제주] 평생 볼 수국, 해바라기 다 보고 온 탐라국 꽃여행  2.000000  2.500000   
...    ...                                   ...       ...       ...   
1411  3804                                제주도 여행  1.500000  1.916667   
1412  3805                          똥강아지패밀리 신혼여행  2.538462  2.923077   
1413  3806                              엄마랑 제주여행  2.352941  2.941176   
1414  3807                                   제주도  2.454545  2.636364   
1415  3808                                 Jeju!  2.000000  2.444444   

            4         5         6         7         8         9   10  \
0     1.500000  1.2000

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


###set 차집합 방향확인

In [ ]:
set1 = set([1,2,3,4,5])
set2 = set([1,2,3,7])
set2 -set1

{7}

###range index 확인

In [ ]:
checking_list = [100,20]
length = len(checking_list)
for i in range(-1,-(length+1),-1):
    now_idx = checking_list[i]
    print(now_idx)

20
100


In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db
try:
  ddb = load_db()
except:
  print("no db error")

##load_course 메소드 수정 : > 를 >=로 바꿔줬다. 경계값 조심하기 / 하드코딩 조심하기 (iteration value는 고정된 값보다 변수를 넣어주도록 하자.)